# ***PRÁCTICO FINAL DE CIENCIA DE DATOS II - Coder House***
Estudiante: **Gonzalo Leonel Gramajo**  
Comisión: **75690**  
Documento: **40441349**  
Año: **2025**

Este notebook corresponde al trabajo final del curso "Fundamentos de ciencia de datos" de Coder House. Está basado en un dataset de ventas de autos rescatado desde Keegle. Este conjunto de datos incluye todas las entradas de vehículos usados ​​dentro de los Estados Unidos en Craigslist.com.
Es importante resaltar que se cuenta con 426 mil lineas y el archivo ocupa 1.45 GB de almacenamiento, por lo que un archivo de tal tamaño no se peude subir a GitHub. Dado esto, el archivo .csv se puede encontrar disponible en Google Drive y obviamente, Kaggle.

### **Dataset**
Used Cars Dataset - Vehicles listings from Craigslist.org.  
Enlace web a Kaggle: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data  
Enlace web a Google Drive: https://drive.google.com/file/d/1uQ_YhqBimI46j5W-EgSwkjZvpFt87ejt/view?usp=sharing

___________________________________________________________________________________________
## **Objetivo**
*Mediante el uso de algortimos de aprendizaje automático, lograr predecir el precio de venta de los vehículos según los datos de entrada odómetro (kilometraje), year (año de fabricación), transmission (trasmisión), fuel (combustible).*

___________________________________________________________________________________________
## **PREGUNTAS DE INTERÉS**

* Qué autor es más popular?
* Qué tan frecuente crean contenido
* Qué autor posee contenido más reciente?
* Qué autor tiene mejor contenido?
    * Los autores fueron mejorando su contenido con el pasar del tiempo?
* Qué autor genera más contenido de Python?
    * Qué autor genera el mejor contenido en Python
    * Cuáles sonn los videos más vistos?
    * Cuál es el mejor rankeado?

___________________________________________________________________________________________
## **IMPORTACIÓN DE MÓDULOS Y CARGA DE DATOS**

In [ ]:
# Herramientas principales
import pandas as pd
import numpy as np
# Módulos de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Carga el archivo CSV en un DataFrame de pandas
file_path = './vehicles.csv' # ruta al archivo CSV
df = pd.read_csv(file_path) # leer el archivo CSV y conseguir un dataframe

df.head(10) # mostrar los primeros 5 valores
df.tail(10) # mostrar los últimos 5 valores


KeyboardInterrupt: 

___________________________________________________________________________________________
## **INGENIERÍA DE ATRIBUTOS (FEATURE ENINEERING)**

___________________________________________________________________________________________
## **CONEXIÓN A API'S**

___________________________________________________________________________________________
## **ANÁLISIS EXPLORATORIO DE DATOS (EDA)**

___________________________________________________________________________________________
## **CONCLUSIÓN**

___________________________________________________________________________________________
## **PRÓXIMAS LÍNEAS**